This notebook contains code to test and generate images for the tutorial.

In [ ]:
from IPython.display import Image, SVG, display
import cv2
import smashcima as sc

In [ ]:
from smashcima.synthesis import PaperSynthesizer, SolidColorPaperSynthesizer


class MyModel(sc.orchestration.BaseHandwrittenModel):
    def register_services(self):
        super().register_services()
        
        # for paper synthesizer use the solid color synth,
        # instead of the texture-quilting default synthesizer
        self.container.interface(
            PaperSynthesizer, # interface
            SolidColorPaperSynthesizer # implementation
        )
    
    def resolve_services(self):
        super().resolve_services()

        # get the paper synthesizer instance
        self.paper_synth: SolidColorPaperSynthesizer \
            = self.container.resolve(PaperSynthesizer)

    def configure_services(self):
        super().configure_services()

        # and configure paper synthesizer's properties
        # (BGRA uint8 format)
        self.paper_synth.color = (187, 221, 234, 255)


model = MyModel()
scene = model("../../docs/assets/tutorial-1/lc5003150.musicxml")

bitmap = scene.render(scene.pages[0])
display(Image(
    data=cv2.imencode(".png", bitmap)[1],
    width=900
))

In [ ]:
import random

model = MyModel()
resolved_rng = model.container.resolve(random.Random)
field_rng = model.rng

assert resolved_rng is field_rng # succeeds!

In [ ]:
my_rng = random.Random(42)

class MyRandomModel(sc.orchestration.BaseHandwrittenModel):
    def register_services(self):
        super().register_services()

        self.container.instance(
            random.Random, # when people ask for this
            my_rng # return this
        )

model = MyRandomModel()
assert model.rng is my_rng # succeeds!